# Annotation Files for Manual Linking
>This notebook facilitates writing out non-linked players to an annotation file - then handling the write out to annotation files.  The files can then be loaded here to process: https://j535d165.github.io/recordlinkage-annotator/  Finally the result file can be loaded back into the j_notebook directory where it is picked up here and added to the Record Links database table.

In [1]:
import pandas as pd
import recordlinkage as rl
from recordlinkage.index import Block
import sqlite3 as sql
import functions as fx
import core_constants as cc

## Load Datasets
> Load the source and target datasets.  Currently it is only written for 247 and Rivals but needs to be extended

In [4]:
dataset_keys = {'sports247': 'Unlinked247', 'rivals': 'UnlinkedRivals'}
for key,value in dataset_keys.items():
    SQL = """SELECT * FROM {}""".format(value)
    vars()[key] = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
    (vars()[key]).index.name = key + '_IDYR'
    (vars()[key])['IDYR'] = (vars()[key]).index.get_level_values(0)

## Create Blockers
> In this case, we are creating blockers on college and year to limit the number of possible matches.

#### !!!This should be extended to take into account fuzzy matching confidence level as well!!!

In [6]:
indexer = rl.BlockIndex(on=['College', 'Year'])
candidate_links = indexer.index(sports247, rivals)

## Create Annotation file
> This is actually terrible code.  Currently I'm manually updating by length.  Need to loop through.  TODO

In [42]:
rl.write_annotation_file(
    "annotate_rivals_14.json",
    candidate_links[7001:7500],
    sports247,
    rivals,
    dataset_a_name="247 Sports",
    dataset_b_name="Rivals")

## Read in Result
> Once you handled the above file in the annotator, the below code will verify that the res

In [2]:
annotation = rl.read_annotation_file("result_14.json")
annotation_dict = (annotation.links).to_flat_index()

In [3]:
print(annotation_dict[0][0])

sethsutton_troy_2005


In [4]:
for record in annotation_dict:
    Values = [record[0], record[1], 2, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()